# Chapter 3
---
This chapter covers
- Understanding tensors, the basic data structure in PyTorch
- Indexing and operating on tensors
- Interoperating with NumPy multidimensional arrays
- Moving computations to the GPU for speed

In [1]:
import torch
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [2]:
# use index to access will create a tensor
a[1], float(a[1])

(tensor(1.), 1.0)

using index will create a different view of the same underlying data  
no new momery will be allocated

---
use named tensors

In [3]:
# add name when creating a tensor
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1463.)
  


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [6]:
# use refine_names to reset or add names of the dimensions
# ... will leave out any number of dimensions
batch_t = torch.rand(2, 3, 5, 5)
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named.shape, batch_named.names

(torch.Size([2, 3, 5, 5]), (None, 'channels', 'rows', 'columns'))

In [10]:
batch_named = batch_named.refine_names('batch_size', ...)
batch_named.names

('batch_size', 'channels', 'rows', 'columns')

align_as will add missing dimensions and change the order of the existing ones to the right order

In [12]:
weights_aligned = weights_named.align_as(batch_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([1, 3, 1, 1]), ('batch_size', 'channels', 'rows', 'columns'))

functions can take the name of the dimensions as input

In [13]:
batch_named.sum('channels')

tensor([[[0.6781, 1.4259, 1.9345, 0.7120, 1.4757],
         [0.3244, 1.9906, 1.2447, 0.7481, 1.3925],
         [1.2836, 0.9621, 1.8143, 1.5092, 1.3286],
         [1.6097, 1.1859, 1.7383, 1.7485, 1.7171],
         [1.9866, 1.9968, 0.9487, 1.6563, 1.3509]],

        [[1.2661, 1.3028, 0.3757, 1.7265, 1.5550],
         [1.5287, 1.5014, 1.4711, 1.0422, 1.3916],
         [1.3402, 1.0065, 1.6874, 1.5723, 2.1249],
         [2.0931, 1.5172, 1.1995, 1.8564, 2.5298],
         [2.0060, 1.6784, 1.2382, 1.0963, 1.4368]]],
       names=('batch_size', 'rows', 'columns'))

### Tensor element types
---
use dtype argument to set the data type of a tensor; float int or bool

In [3]:
import torch

double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)
short_points.dtype, double_points.dtype

(torch.int16, torch.float64)

In [5]:
short_points = torch.ones(10, 2).to(dtype=torch.short)
short_points

tensor([[1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1]], dtype=torch.int16)

storage is the underlaying structure of the tensor in the memory.  
several tensors can be views of the same storage, so that change together if one of them is modified  
storage is one dimensional

In [6]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [7]:
stor = points.storage()
stor[0]

4.0

In [8]:
stor[1] = 2.0
points

tensor([[4., 2.],
        [5., 3.],
        [2., 1.]])

In-place operations: have a trailing underscore like zero_  
they modify the input instead of making copy

In [9]:
a = torch.ones(3, 2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

tensors use metadata to store how to change the shape of storage to get this tensor  
- offset: the beginning index of this tensor inside the storage block it is in
- size: each dimension's length
- stride: the count of index to increase in storage if that dimension's value increase by one

In [11]:
points = torch.tensor([[4, 1], [5, 3], [2, 1]])
points

tensor([[4, 1],
        [5, 3],
        [2, 1]])

In [13]:
points.storage_offset(), points.size(), points.stride()

(0, torch.Size([3, 2]), (2, 1))

In [14]:
second_point = points[1]
second_point.storage_offset()

2

transpose only changes the metadata, but doesn't change the storage or allocate memory

In [15]:
points_t = points.t()  # .transpose() for more than two dimensional tensors
id(points.storage()) == id(points_t.storage())

True

In [16]:
points_t.storage_offset(), points_t.size(), points_t.stride()

(0, torch.Size([2, 3]), (1, 2))

contiguous means the tensor's elements' order is the same as its storage order  
is_contiguous() for checking, and contiguous() for switching

In [17]:
points_t.is_contiguous()

False

In [20]:
points_t.contiguous().storage()   # would change the order of the storage

 4
 5
 2
 1
 3
 1
[torch.LongStorage of size 6]

### Numpy interoperability
---
transform from tensor to numpy array makes no change in the storage, they use the same underlying buffer. So it is basically no cost, and change the numpy array will result in change of the tensor

In [22]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [23]:
points_np[0][0] = 0
points

tensor([[0., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [24]:
points = torch.from_numpy(points_np)   # change backward
points

tensor([[0., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

### Serializing tensors
---
save and load tensors

In [25]:
torch.save(points, './outpoints.t')

In [27]:
points = torch.load('./outpoints.t')
points

tensor([[0., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

can also use HDF5 to save and load the tensors as NumPy arrays

In [33]:
import h5py
f = h5py.File('./points.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())  # the first is a key, treat f as a dict
f.close()

In [37]:
# h5py can read only part of the data from the disk
f = h5py.File('./points.hdf5', 'r')  # the data is on the disk
dset = f['coords']                   # the data is not loaded yet
last_points = dset[-2:]              # load only the last two now
f.close()
last_points

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

---
### Exercises
---

In [39]:
a = torch.tensor(list(range(9)))
a.size(), a.storage_offset(), a.stride()

(torch.Size([9]), 0, (1,))

In [40]:
b = a.view(3, 3)
b.size(), b.storage_offset(), b.stride()

(torch.Size([3, 3]), 0, (3, 1))

In [41]:
id(b.storage()) == id(a.storage())

True

In [44]:
c = b[1:, 1:]
c.size(), c.storage_offset(), c.stride()  # take a look at the stride, (3, 1)
# o o o
# o x x
# o x x 


(torch.Size([2, 2]), 4, (3, 1))

In [48]:
a_sqrt = a.to(torch.float).sqrt_()
a_sqrt

tensor([0.0000, 1.0000, 1.4142, 1.7321, 2.0000, 2.2361, 2.4495, 2.6458, 2.8284])